# DataFrames Spark : Exercice

Passons à un exercice pratique que vous allez traiter individuellement. Il est constitué de questions sur des données boursières, dans ce cas les actions de Walmart pour les années 2012-2017.
R répondez aux questions et effectuez les tâches ci-dessous.

#### Utilisez le fichier walmart_stock.csv pour répondre et compléter les tâches ci-dessous !

#### Démarrer une simple session Spark

In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Walmart').getOrCreate()

#### Charger le fichier CSV du stock Walmart, demander à Spark de déterminer les types de données.

In [2]:
df = spark.read.csv('walmart_stock.csv', header=True, inferSchema=True)

#### Quels sont les noms des colonnes ?

In [3]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### À quoi ressemble le schéma ?

In [4]:
df.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Imprimer les 5 premières lignes.

In [6]:
for i in df.head(5):
    print(i)
    print('\n')

Row(Date=datetime.date(2012, 1, 3), Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996)


Row(Date=datetime.date(2012, 1, 4), Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475)


Row(Date=datetime.date(2012, 1, 5), Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539)


Row(Date=datetime.date(2012, 1, 6), Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922)


Row(Date=datetime.date(2012, 1, 9), Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004)




#### Utilisez describe() pour en savoir plus sur le DataFrame.

In [7]:
df.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Question bonus !
#### Il y a trop de décimales pour la moyenne et l'écart-type dans le dataframe describe(). Formatez les nombres pour qu'ils ne comportent que deux décimales. Prêtez attention aux types de données que .describe() renvoie, nous n'avons pas abordé la façon de faire ce formatage exact, mais nous avons abordé quelque chose de très similaire. [Consultez ce lien pour un indice](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

Si vous êtes bloqué, ne vous inquiétez  ! Nous reviendrons sur la solutionns.

In [8]:
df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)



In [9]:
from pyspark.sql.functions import format_number

In [10]:
df_desc = df.describe()

In [11]:
df_desc.select(
    df_desc['summary'],
    format_number(df_desc['Open'].cast('float'),2).alias('Open'),
    format_number(df_desc['High'].cast('float'),2).alias('High'),
    format_number(df_desc['Low'].cast('float'),2).alias('Low'),
    format_number(df_desc['Close'].cast('float'),2).alias('Close'),
    df_desc['Volume'].cast('int').alias('Volume')
).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519781|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### Créez un nouveau cadre de données avec une colonne appelée HV Ratio qui est le ratio du prix le plus élevé par rapport au volume d'actions échangées pour une journée.

In [12]:
df2 = df.withColumn("HV Ratio", df['High']/df['Volume'])
df2.select('HV Ratio').show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### Quel jour a eu lieu le pic de prix le plus élevé ?

In [13]:
from pyspark.sql.functions import desc
df.orderBy(desc('High')).first()[0]

datetime.date(2015, 1, 13)

In [14]:
df.orderBy(df['High'].desc()).head(1)

[Row(Date=datetime.date(2015, 1, 13), Open=90.800003, High=90.970001, Low=88.93, Close=89.309998, Volume=8215400, Adj Close=83.825448)]

In [15]:
df.orderBy(df['High'].desc()).head(1)[0][0]

datetime.date(2015, 1, 13)

#### Quelle est la moyenne de la colonne Close ?

In [16]:
from pyspark.sql.functions import avg
df.select(avg('Close')).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### Quelles sont les valeurs maximale et minimale de la colonne Volume ?

In [17]:
from pyspark.sql.functions import min, max

In [19]:
df.agg(max('Volume'),min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



In [20]:
df.select(max('Volume'),min('Volume')).show()

+-----------+-----------+
|max(Volume)|min(Volume)|
+-----------+-----------+
|   80898100|    2094900|
+-----------+-----------+



#### Combien de jours le Close a-t-il été inférieur à 60 dollars ?

In [21]:
df.filter('Close < 60').count()

81

In [22]:
df.filter(df['Close']<60).count()

81

#### Quel est le pourcentage de temps pendant lequel le cours le plus élevé a été supérieur à 80 dollars ?
#### En d'autres termes, (Nombre de jours où le cours le plus élevé est supérieur à 80 dollars)/(Nombre total de joursdu jeue de données)

In [23]:
nbjourssup80 = df.filter(df['High'] > 80).count()
nblignes = df.count()
nbjourssup80/nblignes * 100

9.141494435612083

9.141494435612083

#### Quelle est la corrélation de Pearson entre le haut et le volume ?
#### [Indice](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [24]:
from pyspark.sql.functions import corr
df.select(corr((df['High']),(df['Volume']))).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### Quel est le montant maximum par an ?

In [26]:
from pyspark.sql.functions import year
df = df.withColumn('Year', year(df['Date']))
df.groupBy('Year').max('High').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



In [117]:
from pyspark.sql.functions import year
df.withColumn('Year', year(df['Date']))
df.groupBy('Year').max('High').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



#### Quel est le cours de clôture moyen pour chaque mois civil ?
#### En d'autres termes, pour toutes les années, quel est le prix de clôture moyen pour les mois de janvier, février, mars, etc. Votre résultat contiendra une valeur pour chacun de ces mois. 

In [27]:
from pyspark.sql.functions import month
df = df.withColumn('Month', month(df['Date']))
df.groupBy('Month').avg('Close').orderBy('Month').show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



# Excellent travail !